# Store Sales Eda, Modelisation And Prediction Test-set

I thank the authors of these notebooks for their knowledge and help on this kaggle competition (Store Sales - Time Series Forecasting).
 
This is my first Kaggle, and I wanted to excercise myself in the field of time series prediction, as I am a beginner in this field.

This notebook will be presented in 3 parts.
An exploratory analysis on my different variables (allowing to explain the relations of different components on my variable Target "sales"°)
Several modeling analyses 
And a prediction analysis of my Test samples.

https://www.kaggle.com/code/ilyakondrusevich/54-stores-54-models

https://www.kaggle.com/code/andrej0marinchenko/hyperparamaters

https://www.kaggle.com/code/ekrembayar/store-sales-ts-forecasting-a-comprehensive-guide

In [ ]:
# Librairie
# importation librarie
import matplotlib as plt
import matplotlib.pyplot as plt  # For use legend, xlabel, ylabel on plt
import numpy as np
import scipy as sp
import sklearn
from IPython.display import display
import pandas as pd

import seaborn as sns
import missingno as msno
import pandas as pd
import calendar

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.offline as offline
import plotly.graph_objs as go
import plotly.express as px
from pathlib import Path
from warnings import simplefilter

import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import mean_squared_log_error
from xgboost import XGBRegressor
from statistics import mean

In [ ]:
download_dir = Path("../input/store-sales-time-series-forecasting")

In [ ]:
holidays_events = pd.read_csv("../input/store-sales-time-series-forecasting/holidays_events.csv", sep=",")
oil = pd.read_csv("../input/store-sales-time-series-forecasting/oil.csv", sep=",")
stores = pd.read_csv("../input/store-sales-time-series-forecasting/stores.csv", sep=",")
test = pd.read_csv("../input/store-sales-time-series-forecasting/test.csv", sep=",")
train = pd.read_csv("../input/store-sales-time-series-forecasting/train.csv", sep=",")

We have 5 files, we will try to merge these files together. 
By grouping oil, store (a categorical variable that represents the store that sold the product), 
vacations and public holidays (holidays_events) on training and test data.
(The transaction values are unknown on the Test set data (this is logical). Moreover the Target variable "sales" is too dependent on them. This Transaction variable is not a good variable for our prediction analysis).

Our goal is to predict the sales of the different product families sold in the different Favorita stores. There are 54 Favorita stores in this dataset. 
There are 54 stores and my idea is to train the model on the data of each store separately.

# Processing of training data

In [ ]:
train.shape

In [ ]:
holidays_events

In [ ]:
oil_train = train.merge(oil, how="left", on="date")
oil_train_st = oil_train.merge(stores, how="left", on="store_nbr")
oil_train_st_h = oil_train_st.merge(holidays_events, how="left", on="date")

In [ ]:
oil_train_st_h.shape

In [ ]:
oil_train_st_tr_h = oil_train_st_h.rename(
    columns={"type_x": "store_type", "type_y": "ho_or_nt"}
)

 Only the number of orders is kept to avoid duplicates (because for the same day you can get several events)

In [ ]:
dt_Train = oil_train_st_tr_h.drop_duplicates(subset=["id"])
dt_Train.shape

In [ ]:
dt_Train

In [ ]:
dt_Train.date = pd.to_datetime(dt_Train.date)
dt_Train["year"] = dt_Train["date"].dt.year
dt_Train["month"] = dt_Train["date"].dt.month
dt_Train["week"] = dt_Train["date"].dt.isocalendar().week
dt_Train["quarter"] = dt_Train["date"].dt.quarter
dt_Train["day_of_week"] = dt_Train["date"].dt.day_name()

In [ ]:
msno.bar(dt_Train.sample(3000888))

Here, we observe that some variables are missing (Nan value), notably the Holiday_or_not and Dcoilwtico columns.  We will have to clean up these variables for the EDA and modeling analyses.

We assume that if there are no transactions for a product, we can replace the missing values by 0

## For event variables 

In [ ]:
dt_Train[["locale", "locale_name", "description"]] = dt_Train[
    ["locale", "locale_name", "description"]
].replace(np.nan, "")
dt_Train["ho_or_nt"] = dt_Train["ho_or_nt"].replace(np.nan, "Work Day")
dt_Train["transferred"] = dt_Train["transferred"].replace(np.nan, False)

In [ ]:
dt_Train["ho_or_nt"] = dt_Train["ho_or_nt"].replace(np.nan, "Work Day")

### For missing essence data, we will use interpolation techniques to find the missing values 


In [ ]:
dt_Train

In [ ]:
oil["date"] = pd.to_datetime(oil["date"])
oil.head()

In [ ]:
# Interpolate
# Resample
oil = (
    oil.set_index("date")["dcoilwtico"].resample("D").sum().reset_index()
)  # add missing dates and fill NaNs with 0

# Interpolate
oil["dcoilwtico"] = np.where(
    oil["dcoilwtico"] == 0, np.nan, oil["dcoilwtico"]
)  # replace 0 with NaN
oil[
    "dcoil_interpolated"
] = oil.dcoilwtico.interpolate()  # fill NaN values using an interpo method

oil.head(10)

In [ ]:
oil.drop(["dcoilwtico"], axis=1, inplace=True)
dt_Train.drop(["dcoilwtico"], axis=1, inplace=True)
dt_Train = dt_Train.merge(oil, how="left", on="date")

In [ ]:
msno.bar(dt_Train.sample(3000888))

In [ ]:
dt_Train = dt_Train.dropna(subset=["sales"])

In [ ]:
dt_Train.shape

We will do the same thing for the Test set data

In [ ]:
oil = pd.read_csv(download_dir / "oil.csv", sep=",")

oil_test = test.merge(oil, how="left", on="date")
oil_test_st = oil_test.merge(stores, how="left", on="store_nbr")
oil_test_st_h = oil_test_st.merge(holidays_events, how="left", on="date")
oil_test_st_h = oil_test_st_h.rename(
    columns={"type_x": "store_type", "type_y": "ho_or_nt"}
)
dt_test = oil_test_st_h.drop_duplicates(subset=["id"])

dt_test = oil_test_st_h
dt_test.date = pd.to_datetime(dt_test.date)
dt_test["year"] = dt_test["date"].dt.year
dt_test["month"] = dt_test["date"].dt.month
dt_test["week"] = dt_test["date"].dt.isocalendar().week
dt_test["quarter"] = dt_test["date"].dt.quarter
dt_test["day_of_week"] = dt_test["date"].dt.day_name()

dt_test[["locale", "locale_name", "description"]] = dt_test[
    ["locale", "locale_name", "description"]
].replace(np.nan, "")

dt_test["ho_or_nt"] = dt_test["ho_or_nt"].replace(np.nan, "Work Day")
dt_test["transferred"] = dt_test["transferred"].replace(np.nan, False)

oil["date"] = pd.to_datetime(oil["date"])
oil = (
    oil.set_index("date")["dcoilwtico"].resample("D").sum().reset_index()
)  # add missing dates and fill NaNs with 0

# Interpolate
oil["dcoilwtico"] = np.where(
    oil["dcoilwtico"] == 0, np.nan, oil["dcoilwtico"]
)  # replace 0 with NaN
oil[
    "dcoil_interpolated"
] = oil.dcoilwtico.interpolate()  # fill NaN values using interpolation method
oil.drop(["dcoilwtico"], axis=1, inplace=True)
dt_test.drop(["dcoilwtico"], axis=1, inplace=True)
dt_test = dt_test.merge(oil, how="left", on="date")

In [ ]:
dt_test

In [ ]:
msno.bar(dt_test.sample(28512))

In [ ]:
# Save DataFrame test
dt_test.to_csv("TestFinish.csv", index=False)

# Let's move on to the exploratory analysis step 

How many cities are represented in the dataset

## Spatial analysis 

In [ ]:
df2 = (
    dt_Train.groupby(by=["state", "city"])
    .agg({"sales": "mean"})
    .reset_index()
    .sort_values(by="sales", ascending=False)
)

In [ ]:
# Histogram representing the sale of products from each city by state
fig = px.bar(df2, x="state", y="sales", color="city", text_auto=True)
fig.update_layout(
    autosize=False, width=1000, height=800,
)
fig.show()

4 states (Pichincha, Guayas, Los rios, Manabi, for which they have several cities) have a higher sales rate than the other states.
 And 2 big cities have a sales rate higher than 500

In [ ]:
dfcl = (
    dt_Train.groupby(by=["cluster"])
    .agg({"sales": "mean"})
    .reset_index()
    .sort_values(by="sales", ascending=False)
)

In [ ]:
# Histogram representing the sale by cluster
fig = px.bar(dfcl, x="cluster", y="sales")
fig.update_layout(
    autosize=False, width=1000, height=800,
)
fig.show()

Cluster n°5 seems to stand out from the other clusters with a sales value above 1000

In [ ]:
dfc2 = (
    dt_Train.groupby(by=["state", "cluster"])
    .agg({"sales": "mean"})
    .reset_index()
    .sort_values(by="sales", ascending=False)
)

# Histogram representing the sale of each cluster by state
fig = px.bar(dfc2, x="state", y="sales", color="cluster", text_auto=True)
fig.update_layout(
    autosize=False, width=1000, height=800,
)
fig.show()

In [ ]:
dfc2 = (
    dt_Train.groupby(by=["city", "cluster"])
    .agg({"sales": "mean"})
    .reset_index()
    .sort_values(by="sales", ascending=False)
)

# Histogram representing the sale of each cluster by state
fig = px.bar(dfc2, x="city", y="sales", color="cluster", text_auto=True)
fig.update_layout(
    autosize=False, width=1000, height=800,
)
fig.show()

 Here we observe that clusters are not specifically grouped according to geography

## Types of stores

In [ ]:
dt_Train.dtypes

In [ ]:
df3 = (
    dt_Train.groupby("store_type")
    .agg({"sales": "mean"})
    .reset_index()
    .sort_values(by="sales", ascending=False)
)

In [ ]:
fig = px.pie(df3, values="sales", names="store_type", hole=0.5)
fig.update_layout(
    autosize=False, width=1000, height=800,
)
fig.show()

In [ ]:
df4 = (
    dt_Train.groupby(by=["store_type", "family"])
    .agg({"sales": "mean"})
    .reset_index()
    .sort_values(by="sales", ascending=False)
)

# Histogram representing the sale of each family product by store type
fig = px.bar(df4, x="store_type", y="sales", color="family", text_auto=True)
fig.update_layout(
    autosize=False, width=1000, height=800,
)
fig.show()

## Sales analysis by product (familly) group

In [ ]:
dt_Train["family"].unique().tolist()

In [ ]:
# There are 33 product groups

In [ ]:
dff = (
    dt_Train.groupby("family")
    .agg({"sales": "mean"})
    .reset_index()
    .sort_values(by="sales", ascending=False)
)

In [ ]:
fig = px.bar(dff, x="family", y="sales", text_auto=True)
fig.update_layout(
    autosize=False, width=1000, height=1000,
)
fig.show()

the group with the highest sales value is Grocery 1

## Event analysis

In [ ]:
dff = (
    dt_Train.groupby("ho_or_nt")
    .agg({"sales": "mean"})
    .sort_values(by="sales", ascending=False)
)

In [ ]:
Eventbyday = dt_Train.drop_duplicates(subset=["date"])
sns.catplot(x="ho_or_nt", kind="count", palette="ch:.25", data=Eventbyday)

In [ ]:
dff.plot(kind="bar", figsize=(12, 6)).set_title("average holiday sales")

 It seems that during the periods outside of work, there is slight increase in sales 
    Exception for New Year's Day and additional day of holiday (making Christmas Eve a holiday)

## Oil Analysis 

Why the oil ? 
Ecuador is a small "dollarized" economy.
It is relatively open, very dependent on oil revenues (which represent between 14% and 20% of its GDP, more than half of export revenues, and between 20% and 30% of government revenues)


In [ ]:
dfoil = (
    dt_Train.groupby(by=["city", "date"])
    .agg({"dcoil_interpolated": "mean"})
    .reset_index()
    .sort_values(by="dcoil_interpolated", ascending=False)
)

In [ ]:
sns.relplot(
    x="date",
    y="dcoil_interpolated",
    hue="city",
    estimator=None,
    kind="line",
    data=dfoil,
)

There was a sudden drop in oil prices during 2014. The price of gasoline is similar between the different cities.

 There seems to be a relationship between the different product groups and the oil price
In general, the different product groups have the same character as the oil price 
we observe two subsets with correlations (Spearman) more or less strong depending on the product.
The first subset includes more sales and corresponds to an oil price below 70. And the second group containing less sales corresponding to an oil price above 70.


In [ ]:
oil = pd.read_csv(download_dir / "oil.csv", sep=",")

# Interpolate
oil["dcoilwtico"] = np.where(
    oil["dcoilwtico"] == 0, np.nan, oil["dcoilwtico"]
)  # replace 0 with NaN
oil[
    "dcoil_interpolated"
] = oil.dcoilwtico.interpolate()  # fill NaN values using interpolation

oil.head(10)

In [ ]:
def plot_sales_and_oil_dependency():
    a = pd.merge(
        train.groupby(["date", "family"]).sales.sum().reset_index(),
        oil.drop("dcoilwtico", axis=1),
        how="left",
    )
    c = a.groupby("family").corr("spearman").reset_index()
    c = c[c.level_1 == "dcoil_interpolated"][["family", "sales"]].sort_values("sales")

    fig, axes = plt.subplots(7, 5, figsize=(20, 20))
    for i, fam in enumerate(c.family):
        a[a.family == fam].plot.scatter(
            x="dcoil_interpolated", y="sales", ax=axes[i // 5, i % 5]
        )
        axes[i // 5, i % 5].set_title(
            fam + "\n Correlation:" + str(c[c.family == fam].sales.iloc[0])[:6],
            fontsize=12,
        )
        axes[i // 5, i % 5].axvline(x=70, color="r", linestyle="--")

    plt.tight_layout(pad=5)
    plt.suptitle("Daily Oil Product & Total Family Sales \n", fontsize=20)
    plt.show()


plot_sales_and_oil_dependency()

# Time analysis of product sales 

In [ ]:
dffa = (
    dt_Train.groupby(by=["family", "date"])
    .agg({"sales": "mean"})
    .reset_index()
    .sort_values(by="date", ascending=True)
)

In [ ]:
px.line(
    dffa, x="date", y="sales", color="family", title="Daily total sales of the family"
)

It can be observed that there are certain product groups that follow a growing trend over the years (like grocery groups or bakery).They also follow daily, weekly and monthly variations and during certain periods of the year (increase in sales during December).
But other products (lingerie, book) do not follow annual, seasonal, monthly or weekly trends.


In [ ]:
dffaf = (
    dt_Train.set_index("date")
    .groupby("store_nbr")
    .resample("D")
    .sales.sum()
    .reset_index()
)
px.line(
    dffaf,
    x="date",
    y="sales",
    color="store_nbr",
    title="Daily total sales of the stores",
)

For the total daily sales of the stores, we observe that the stores follow the same temporal trends but with variations on the quantity of product sold 

I will aggregate the data by stores and families on my sales values. To give an average sales index and facilitate the visualization of the results.

In [ ]:
dt_Train.dtypes

In [ ]:
Av_sale_Ye = (
    dt_Train.groupby("year")
    .agg({"sales": "mean"})
    .reset_index()
    .sort_values(by="sales", ascending=False)
)

In [ ]:
sns.catplot(x="year", y="sales", data=Av_sale_Ye, kind="bar")

In [ ]:
Av_sale_q = (
    dt_Train.groupby("quarter")
    .agg({"sales": "mean"})
    .reset_index()
    .sort_values(by="sales", ascending=False)
)

In [ ]:
sns.catplot(x="quarter", y="sales", data=Av_sale_q, kind="bar")

In [ ]:
Av_sale_M = (
    dt_Train.groupby("month")
    .agg({"sales": "mean"})
    .reset_index()
    .sort_values(by="sales", ascending=False)
)

In [ ]:
sns.catplot(x="month", y="sales", data=Av_sale_M, kind="bar")

In [ ]:
Av_sale_W = (
    dt_Train.groupby("week")
    .agg({"sales": "mean"})
    .reset_index()
    .sort_values(by="sales", ascending=False)
)

In [ ]:
fig = px.bar(Av_sale_W, x="week", y="sales")
fig.show()

In [ ]:
Av_sale_D = (
    dt_Train.groupby("day_of_week")
    .agg({"sales": "mean"})
    .reset_index()
    .sort_values(by="sales", ascending=False)
)

In [ ]:
sns.catplot(x="day_of_week", y="sales", data=Av_sale_D, kind="bar")

On this dataset, the average sales per product and per shop follow a trend over the years (increase in sales rates per year), a quarterly variability marked by certain periods of the year which will increase the purchase rate, a variability at the beginning and end of the month where sales increase (buyers' income at the end of the month) and a daily variability where on Saturdays and Sundays, there is an overall increase in sales over the week. 



# Let's move on to the modeling stage 

Before predicting the test values, we will build several models to improve the accuracy of the model. The metric used will be the RMSE 

Why RMSE ?

It is often used to predict time series and has many advantages. It is robust to outliers (we have not treated outliers). It calculates the relative error between the predicted and actual values and penalises underestimation of the actual value more severely than overestimation. In general, unlike RMSE (Root Mean Squared Error), it performs well with labels that can take on values over several orders of magnitude.

We can see document of RMSE:
https://www.kaggle.com/general/215997 of Satish Gunjal
Or Very good Notebook on Model Fit Metrics 
https://www.kaggle.com/code/residentmario/model-fit-metrics/notebook of Aleksey Bilogur




We will use the functions of the course (Kaggle Time Series) to build our model 

In [ ]:
simplefilter("ignore")  # ignore warnings to clean up output cells

# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True, figsize=(11, 5))
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)
plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
    legend=False,
)
%config InlineBackend.figure_format = 'retina'

In [ ]:
def fourier_features(index, freq, order):
    time = np.arange(len(index), dtype=np.float32)
    k = 2 * np.pi * (1 / freq) * time
    features = {}
    for i in range(1, order + 1):
        features.update(
            {f"sin_{freq}_{i}": np.sin(i * k), f"cos_{freq}_{i}": np.cos(i * k)}
        )
    return pd.DataFrame(features, index=index)


# Compute Fourier features to the 4th order (8 new features) for a
# series y with daily observations and annual seasonality:
#
# fourier_features(y, freq=365.25, order=4)

# annotations: https://stackoverflow.com/a/49238256/5769929
def seasonal_plot(X, y, period, freq, ax=None):
    if ax is None:
        _, ax = plt.subplots()
    palette = sns.color_palette("husl", n_colors=X[period].nunique())
    ax = sns.lineplot(
        x=freq, y=y, hue=period, data=X, ci=False, ax=ax, palette=palette, legend=False,
    )
    ax.set_title(f"Seasonal Plot ({period}/{freq})")
    for line, name in zip(ax.lines, X[period].unique()):
        y_ = line.get_ydata()[-1]
        ax.annotate(
            name,
            xy=(1, y_),
            xytext=(6, 0),
            color=line.get_color(),
            xycoords=ax.get_yaxis_transform(),
            textcoords="offset points",
            size=14,
            va="center",
        )
    return ax


def plot_periodogram(ts, detrend="linear", ax=None):
    from scipy.signal import periodogram

    fs = pd.Timedelta("1Y") / pd.Timedelta("1D")
    freqencies, spectrum = periodogram(
        ts, fs=fs, detrend=detrend, window="boxcar", scaling="spectrum",
    )
    if ax is None:
        _, ax = plt.subplots()
    ax.step(freqencies, spectrum, color="purple")
    ax.set_xscale("log")
    ax.set_xticks([1, 2, 4, 6, 12, 26, 52, 104])
    ax.set_xticklabels(
        [
            "Annual (1)",
            "Semiannual (2)",
            "Quarterly (4)",
            "Bimonthly (6)",
            "Monthly (12)",
            "Biweekly (26)",
            "Weekly (52)",
            "Semiweekly (104)",
        ],
        rotation=30,
    )
    ax.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))
    ax.set_ylabel("Variance")
    ax.set_title("Periodogram")
    return ax

To build our model we will only use the year 2017.
Because we will predict that 15 days of data following the Train set 

For our model, we will apply a foursquare feature to capture the overall shape of the curves 
seasonal, quarterly, monthly and daily.

In [ ]:
Average_sales = (
    dt_Train.groupby(by=["date"])
    .agg({"sales": "mean"})
    .reset_index()
    .sort_values(by="date", ascending=True)
)

In [ ]:
family_sales = (
    dt_Train.groupby(by=["family", "date"])
    .agg({"sales": "mean"})
    .reset_index()
    .sort_values(by="date", ascending=True)
)

In [ ]:
family_sales2 = (
    family_sales.groupby(["family", "date"])
    .mean()
    .unstack("family")
    .loc["2017", ["sales"]]
)

In [ ]:
food_sales1 = family_sales2.sales["GROCERY I"]

How many Fourier pairs should we actually include in our feature set? We can answer this question with the periodogram. The periodogram tells you the strength of the frequencies in a time series.

In [ ]:
plot_periodogram(food_sales1)

The periodogram agrees with the seasonal plots above: a strong weekly season and a weaker mensual season. The weekly season we'll model with indicators and the annual season with Fourier features. 
The periodogram indicates that we should take a value Weekly. 
We'll create our seasonal features using DeterministicProcess

In [ ]:
dt_Train["store_nbr"] = pd.Categorical(dt_Train["store_nbr"])
dt_Train["day_of_week"] = pd.Categorical(dt_Train["day_of_week"])
dt_Train["city"] = pd.Categorical(dt_Train["city"])
dt_Train["cluster"] = pd.Categorical(dt_Train["cluster"])
dt_Train["store_type"] = pd.Categorical(dt_Train["store_type"])
dt_Train["state"] = pd.Categorical(dt_Train["state"])
dt_Train["city"] = pd.Categorical(dt_Train["city"])
dt_Train["family"] = pd.Categorical(dt_Train["family"])
dt_Train["ho_or_nt"] = pd.Categorical(dt_Train["ho_or_nt"])
dt_Train["description"] = pd.Categorical(dt_Train["description"])

In [ ]:
# Save Data train
dt_Train.to_csv("TrainFinish.csv", index=False)

In [ ]:
# First simple Build Model

In [ ]:
download_dir2 = Path("TrainFinish.csv")

In [ ]:
store_sales = pd.read_csv("TrainFinish.csv",
    usecols=["store_nbr", "family", "date", "sales"],
    dtype={"store_nbr": "float32", "family": "category", "sales": "float32"},
    parse_dates=["date"],
    infer_datetime_format=True,
)

store_sales = store_sales.dropna(subset=["sales"])

store_sales["store_nbr"] = store_sales[
    "store_nbr"].astype(float).astype(int)

store_sales["store_nbr"] = pd.Categorical(store_sales["store_nbr"])

store_sales["date"] = store_sales.date.dt.to_period("D")

store_sales = store_sales.set_index([
    "store_nbr", "family", "date"]).sort_index()

average_sales = store_sales.groupby("date").mean().squeeze().loc["2017"]

In [ ]:
store_sales

In [ ]:
y = store_sales.unstack(["store_nbr", "family"]).loc["2017"]

In [ ]:
# Create training data

fourier = CalendarFourier(freq="W", order=4)
dp = DeterministicProcess(
    index=y.index,
    constant=True,
    order=1,
    seasonal=False,
    additional_terms=[fourier],
    drop=True,
)
X = dp.in_sample()

In [ ]:
X

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.3, random_state=1, shuffle=False
)

model = LinearRegression(fit_intercept=False)
model.fit(X_train, y_train)

In [ ]:
y_fit = pd.DataFrame(
    model.predict(X_train), index=X_train.index, columns=y_train.columns
).clip(0.0)

y_pred = pd.DataFrame(
    model.predict(X_val), index=X_val.index, columns=y_val.columns
).clip(0.0)

In [ ]:
from sklearn.metrics import mean_squared_log_error

In [ ]:
rmsle_train = mean_squared_log_error(y_train, y_fit) ** 0.5
rmsle_valid = mean_squared_log_error(y_val, y_pred) ** 0.5
print(f"Training RMSLE: {rmsle_train:.5f}")
print(f"Validation RMSLE: {rmsle_valid:.5f}")

Why RMSLE ?
the RMSLE is robust to outliers(We did not treat the Outliers). 
It calculates the relative error between predicted and actual values, and penalizes underestimation of the actual value more severely than overestimation.

In [ ]:
y.columns[0:34]

In [ ]:
families = y.columns[0:6]
axs = y.loc(axis=1)[families].plot(
    subplots=True,
    sharex=True,
    figsize=(11, 9),
    **plot_params, alpha=0.5
)

_ = y_fit.loc(axis=1)[families].plot(
    subplots=True,
    sharex=True,
    color="C0",
    ax=axs)

_ = y_pred.loc(axis=1)[families].plot(
    subplots=True,
    sharex=True,
    color="C3",
    ax=axs)

for ax, family in zip(axs, families):
    ax.legend([])
    ax.set_ylabel(family)

We can do better by adding several variables 

In [ ]:
# Create training data
fourier = CalendarFourier(freq="W", order=4)
dp = DeterministicProcess(
    index=y.index,
    constant=True,
    order=1,
    seasonal=False,
    additional_terms=[fourier],
    drop=True,
)
X = dp.in_sample()
X["NewYear"] = X.index.dayofyear == 1

In [ ]:
X

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.3, random_state=1, shuffle=False
)

model = LinearRegression(fit_intercept=False)
model.fit(X_train, y_train)

y_fit = pd.DataFrame(
    model.predict(X_train), index=X_train.index, columns=y_train.columns
).clip(0.0)

y_pred = pd.DataFrame(
    model.predict(X_val), index=X_val.index, columns=y_val.columns
).clip(0.0)
rmsle_train = mean_squared_log_error(y_train, y_fit) ** 0.5
rmsle_valid = mean_squared_log_error(y_val, y_pred) ** 0.5
print(f"Training RMSLE: {rmsle_train:.5f}")
print(f"Validation RMSLE: {rmsle_valid:.5f}")

In [ ]:
families = y.columns[0:6]
axs = y.loc(axis=1)[families].plot(
    subplots=True,
    sharex=True,
    figsize=(11, 9),
    **plot_params, alpha=0.5
)

_ = y_fit.loc(axis=1)[families].plot(
    subplots=True,
    sharex=True,
    color="C0",
    ax=axs)

_ = y_pred.loc(axis=1)[families].plot(
    subplots=True,
    sharex=True,
    color="C3",
    ax=axs)

for ax, family in zip(axs, families):
    ax.legend([])
    ax.set_ylabel(family)

# I will select the variables useful for this study 

In [ ]:
store_sales1 = pd.read_csv("TrainFinish.csv",
    usecols=[
        "store_nbr",
        "family",
        "date",
        "day_of_week",
        "city",
        "ho_or_nt",
        "description",
        "dcoil_interpolated",
        "year",
        "month",
        "week",
        "quarter",
        "sales",
    ],
    dtype={
        "store_nbr": "float32",
        "family": "category",
        "sales": "float32",
        "day_of_week": "category",
        "city": "category",
        "ho_or_nt": "category",
        "description": "category",
    },
    parse_dates=["date"],
    infer_datetime_format=True,
)
store_sales1 = store_sales1.dropna(subset=["sales"])
store_sales1["store_nbr"] = store_sales1["store_nbr"].astype(float).astype(int)
store_sales1["store_nbr"] = pd.Categorical(store_sales1["store_nbr"])

store_sales1["date"] = store_sales1.date.dt.to_period("D")

store_sales1 = store_sales1.set_index(["store_nbr",
                                       "family", "date"]).sort_index()

In [ ]:
s = store_sales1.reset_index()
s = s.loc[(s["date"] >= "2017-01-01")]
s.drop(
    ["store_nbr", "family", "sales", "city", "description", "year"],
    axis=1,
    inplace=True,
)
s = s.drop_duplicates(subset=["date"])

In [ ]:
s

In [ ]:
transformer = make_column_transformer(
    (OneHotEncoder(sparse=False), ["ho_or_nt", "day_of_week"]), remainder="passthrough",
)

In [ ]:
transformer

In [ ]:
transformed = transformer.fit_transform(s)
transformed_df = pd.DataFrame(transformed,
                              columns=transformer.get_feature_names())

In [ ]:
transformed_df

In [ ]:
transformed_df.dtypes

I have to set my variables to float 

In [ ]:
X_variable = transformed_df.set_index(["date"])
X_variable = X_variable.astype(float)

And I merge my data array with the variable X (created by DeterministicProcess)

In [ ]:
X2 = X.merge(X_variable, how="left", on="date")

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X2, y, test_size=0.3, random_state=1, shuffle=False
)

model = LinearRegression(fit_intercept=False)
model.fit(X_train, y_train)

In [ ]:
y_fit = pd.DataFrame(
    model.predict(X_train), index=X_train.index, columns=y_train.columns
).clip(0.0)

y_pred = pd.DataFrame(
    model.predict(X_val), index=X_val.index, columns=y_val.columns
).clip(0.0)

In [ ]:
rmsle_train = mean_squared_log_error(y_train, y_fit) ** 0.5
rmsle_valid = mean_squared_log_error(y_val, y_pred) ** 0.5
print(f"Training RMSLE: {rmsle_train:.5f}")
print(f"Validation RMSLE: {rmsle_valid:.5f}")

In [ ]:
families = y.columns[0:6]
axs = y.loc(axis=1)[families].plot(
    subplots=True,
    sharex=True,
    figsize=(11, 9),
    **plot_params, alpha=0.5)

_ = y_fit.loc(axis=1)[families].plot(subplots=True,
                                     sharex=True,
                                     color="C0",
                                     ax=axs)

_ = y_pred.loc(axis=1)[families].plot(subplots=True,
                                      sharex=True,
                                      color="C3",
                                      ax=axs)
for ax, family in zip(axs, families):
    ax.legend([])
    ax.set_ylabel(family)

The results of the validation seem less good but more robust 

### We will try another regression model

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X2, y, test_size=0.3, random_state=1, shuffle=False
)

In [ ]:
param_grid = {"ridge__alpha": [0.01, 0.1, 1, 10, 20, 100, 200]}
pipe = make_pipeline(RobustScaler(), Ridge())

# Dont use RMSLE, problem that needs to be solved ,

grid = GridSearchCV(pipe, param_grid, scoring="neg_mean_absolute_error", cv=5)

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
print("Training-set Score : {:.2f}".format(grid.score(X_train, y_train)))
print(" Test-set Score : {:.2f}".format(grid.score(X_val, y_val)))
print("MAE : {:.2f}".format(grid.best_score_))
print("Best parameters: {}".format(grid.best_params_))
y_pred = grid.predict(X_val).clip(0.0)
print("mean_squared_log_error: %.2f" % mean_squared_log_error(y_val, y_pred))

# calculate the RMSLE on the test set

rmsle = np.sqrt(metrics.mean_squared_log_error(y_val, y_pred))
print("rmsle: {:.2f}".format(rmsle))

R2 compares the fit of the chosen model with that of a horizontal straight line (the null hypothesis). If the chosen model fits worse than a horizontal line, then R2 is negative. Note that R2 is not always the square of anything, so it can have a negative value without violating any rules of math. R2

is negative only when the chosen model does not follow the trend of the data, so fits worse than a horizontal line.

In [ ]:
y_fit = pd.DataFrame(
    grid.predict(X_train), index=X_train.index, columns=y_train.columns
).clip(0.0)
y_pred = pd.DataFrame(
    grid.predict(X_val), index=X_val.index, columns=y_val.columns
).clip(0.0)

In [ ]:
rmsle_train = mean_squared_log_error(y_train, y_fit) ** 0.5
rmsle_valid = mean_squared_log_error(y_val, y_pred) ** 0.5
print(f"Training RMSLE: {rmsle_train:.5f}")
print(f"Validation RMSLE: {rmsle_valid:.5f}")

In [ ]:
families = y.columns[0:6]
axs = y.loc(axis=1)[families].plot(
    subplots=True, sharex=True, figsize=(11, 9), **plot_params, alpha=0.5,
)
_ = y_fit.loc(axis=1)[families].plot(subplots=True,
                                     sharex=True, color="C0", ax=axs)
_ = y_pred.loc(axis=1)[families].plot(subplots=True,
                                      sharex=True, color="C3", ax=axs)

for ax, family in zip(axs, families):
    ax.legend([])
    ax.set_ylabel(family)

The result seems almost identical to the previous one 

however, we did not use the variable Onpromotion which is correlated to the target variable.

In [ ]:
stores.index

Let's change slightly the structure of the dataset. We will use a loop for each store, 54 stores.

In [ ]:
stores = pd.read_csv(download_dir / "stores.csv", sep=",")

In [ ]:
dt_Train = pd.read_csv("TrainFinish.csv",
    usecols=[
        "store_nbr",
        "family",
        "date",
        "onpromotion",
        "sales",
        "dcoil_interpolated",
        "city",
    ],
    dtype={"family": "object",
           "onpromotion": "int32",
           "store_nbr": "category"},
    parse_dates=["date"],
    infer_datetime_format=True,
)

In [ ]:
dt_Train["date"] = dt_Train.date.dt.to_period("D")

In [ ]:
dt_Train["store_nbr"] = pd.Categorical(dt_Train["store_nbr"])

In [ ]:
dt_Train

In [ ]:
stores = stores.set_index(["store_nbr"]).sort_index()

In [ ]:
train_rdy = dt_Train.set_index(["store_nbr", "family", "date"]).sort_index()
train_rdy = train_rdy.dropna(subset=["sales"])
display(train_rdy)

In [ ]:
stores.index

In [ ]:
sdate = "2017-01-01"  # start and end of training date
edate = "2017-08-15"
y_arr = []
onpromotion_arr = []

for nbr in stores.index:
    # y_arr
    temp = train_rdy.loc[str(nbr), "sales"]
    y_arr.append(temp.unstack(["family"]).loc[sdate:edate])

    # onpromotion_arr
    onpromotion = train_rdy.loc[str(nbr), "onpromotion"]
    onpromotion = onpromotion.unstack(["family"]).loc[sdate:edate].sum(axis=1)
    onpromotion.name = "onpromotion"
    onpromotion_arr.append(onpromotion)

y_arr[0]  # sales of store_nbr 2

In [ ]:
fourier = CalendarFourier(freq="W", order=4)
X_arr = []
X_test_r = []
store_index = 1

for y, onpromotion in zip(y_arr, onpromotion_arr):
    dp = DeterministicProcess(
        index=y.index,
        constant=True,
        order=1,
        seasonal=False,
        additional_terms=[fourier],
        drop=True,
    )

    X = dp.in_sample()
    X = X.merge(onpromotion, how="left", left_index=True, right_index=True)
    X = X.merge(X_variable, how="left", on="date")

    store_state = stores.loc[store_index, "state"]
    store_city = stores.loc[store_index, "city"]

    X_arr.append(X)

In [ ]:
ridge = make_pipeline(RobustScaler(), Ridge(alpha=10))

# RegressorChain(base_estimator=XGBRegressor()) #rmsle 0.53
# ridge = make_pipeline(RobustScaler(),XGBRegressor())

# ridge = make_pipeline(RobustScaler(),
# RegressorChain(base_estimator=XGBRegressor()))
# Training_average RMSLE: 0.00092
# Validation_average RMSLE: 0.51607

Xgboost cannot extrapolate, XGBoost predictions are only based on a sum of values attached to the leaves of the tree. No transformation is applied to these values: no scaling, no log, no exponential, nothing.

This means that XGBoost can only make a good prediction for situations already encountered in the training history. It will not be able to capture trends!

In [ ]:
rmsle_val_r = []
rmsle_train_r = []
y_pred_r = []
y_fit_r = []
rmsle_val_Fa = []  # rmsle family product

for X, y in zip(X_arr, y_arr):

    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.3, random_state=1, shuffle=False
    )

    model = ridge.fit(X_train, y_train)
    # y_pred = pd.DataFrame(model.predict(X), index=X.index,
    # columns=y.columns).clip(0.0)

    y_fit = pd.DataFrame(
        model.predict(X_train), index=X_train.index, columns=y_train.columns
    ).clip(0.0)
    y_fit_r.append(y_fit)

    y_pred = pd.DataFrame(
        model.predict(X_val), index=X_val.index, columns=y_val.columns
    ).clip(0.0)
    y_pred_r.append(y_pred)

    rmsle_train = mean_squared_log_error(y_train, y_fit) ** 0.5
    rmsle_train_r.append(rmsle_train)

    rmsle_val = mean_squared_log_error(y_val, y_pred) ** 0.5
    rmsle_val_r.append(rmsle_val)

    y_val = y_val.stack(["family"]).reset_index()
    y_val["pred"] = y_pred.stack(["family"]).reset_index().loc[:, 0]

    rmsle_val_Fa.append(
        y_val.groupby("family").apply(
            lambda r: mean_squared_log_error(r.loc[:, 0], r["pred"])
        )
    )

In [ ]:
# For exemple store 1
pd.DataFrame(rmsle_val_Fa[0])

In [ ]:
rmsle_train = mean(rmsle_train_r)
rmsle_valid = mean(rmsle_val_r)
print(f"Training_average RMSLE: {rmsle_train:.5f}")
print(f"Validation_average RMSLE: {rmsle_valid:.5f}")

Correct result !!
The optimization of the hyperparameter (alpha) was done manually. 
The city and town variable did not improve the performance of the model 

# Let's move on to the Prediction data 

with the best model (ridge regression)

In [ ]:
dt_test = pd.read_csv("TestFinish.csv",
    usecols=["store_nbr", "family",
             "date", "onpromotion",
             "dcoil_interpolated"],
    dtype={"family": "object",
           "onpromotion": "int32",
           "store_nbr": "category"},
    parse_dates=["date"],
    infer_datetime_format=True,
)

In [ ]:
dt_Train = pd.read_csv("TrainFinish.csv",
    usecols=[
        "store_nbr",
        "family",
        "date",
        "onpromotion",
        "sales",
        "dcoil_interpolated",
    ],
    dtype={"family": "object",
           "onpromotion": "int32",
           "store_nbr": "category"},
    parse_dates=["date"],
    infer_datetime_format=True,
)

In [ ]:
dt_Train["date"] = dt_Train.date.dt.to_period("D")
dt_test["date"] = dt_test.date.dt.to_period("D")

dt_Train["store_nbr"] = pd.Categorical(dt_Train["store_nbr"])
dt_test["store_nbr"] = pd.Categorical(dt_test["store_nbr"])

train_rdy = dt_Train.set_index(["store_nbr", "family", "date"]).sort_index()
train_rdy = train_rdy.dropna(subset=["sales"])

test_rdy = dt_test.set_index(["store_nbr", "family", "date"]).sort_index()

In [ ]:
Va_test = pd.read_csv("TestFinish.csv",
    usecols=[
        "ho_or_nt",
        "month",
        "date",
        "week",
        "dcoil_interpolated",
        "quarter",
        "day_of_week",
    ],
    dtype={"day_of_week": "category", "holiday_or_not": "category"},
    parse_dates=["date"],
    infer_datetime_format=True,
)

s = Va_test.reset_index()
s.drop(["index"], axis=1, inplace=True)

In [ ]:
s["date"] = s.date.dt.to_period("D")
s = s.drop_duplicates(subset=["date"])

In [ ]:
s

In [ ]:
transformed_df.dtypes

In [ ]:
# Creation of X_variable_test

transformer = make_column_transformer(
    (OneHotEncoder(sparse=False), ["ho_or_nt",
                                   "day_of_week"]),
    remainder="passthrough"
)

transformed = transformer.fit_transform(s)
transformed_df = pd.DataFrame(transformed,
                              columns=transformer.get_feature_names())

X_variable_test = transformed_df.set_index(["date"])
X_variable_test = X_variable_test.astype(float)

In [ ]:
X_variable_test

In [ ]:
X_variable

In [ ]:
X_variable.dtypes

The data_test is missing 3 features:
onehotencoder__x0_Additional
onehotencoder__x0_Event
onehotencoder__x0_Transfer
They need to be added.


In [ ]:
X_variable_test = X_variable_test.assign(
    onehotencoder__x0_Additional=0,
    onehotencoder__x0_Transfer=0,
    onehotencoder__x0_Event=0,
)
X_variable_test = X_variable_test[
    [
        "onehotencoder__x0_Additional",
        "onehotencoder__x0_Event",
        "onehotencoder__x0_Holiday",
        "onehotencoder__x0_Transfer",
        "onehotencoder__x0_Work Day",
        "onehotencoder__x1_Friday",
        "onehotencoder__x1_Monday",
        "onehotencoder__x1_Saturday",
        "onehotencoder__x1_Sunday",
        "onehotencoder__x1_Thursday",
        "onehotencoder__x1_Tuesday",
        "onehotencoder__x1_Wednesday",
        "month",
        "week",
        "quarter",
        "dcoil_interpolated",
    ]
]


X_variable_test = X_variable_test.astype(float)

In [ ]:
sdate = "2017-01-01"  # start and end of training date
edate = "2017-08-15"
y_arr = []
onpromotion_train = []
onpromotion_test = []

for nbr in stores.index:
    # y_arr
    temp = train_rdy.loc[str(nbr), "sales"]
    y_arr.append(temp.unstack(["family"]).loc[sdate:edate])

    # onpromotion_Train
    onpromotion = train_rdy.loc[str(nbr), "onpromotion"]
    onpromotion = onpromotion.unstack(["family"]).loc[sdate:edate].sum(axis=1)
    onpromotion.name = "onpromotion"
    onpromotion_train.append(onpromotion)

    # onpromotion_Test
    test_onpromotion = test_rdy.loc[str(nbr), "onpromotion"]
    test_onpromotion = test_onpromotion.unstack(["family"]).sum(axis=1)
    test_onpromotion.name = "onpromotion"
    onpromotion_test.append(test_onpromotion)

In [ ]:
fourier = CalendarFourier(freq="W", order=4)
X_arr = []
X_test_r = []

for y, onpromotion_Tr, onpromotion_Te in zip(
    y_arr, onpromotion_train, onpromotion_test
):

    dp = DeterministicProcess(
        index=y.index,
        constant=True,
        order=1,
        seasonal=False,
        additional_terms=[fourier],
        drop=True,
    )

    X = dp.in_sample()
    X = X.merge(onpromotion_Tr, how="left",
                left_index=True, right_index=True)
    X = X.merge(X_variable, how="left", on="date")

    X_arr.append(X)

    X_test = dp.out_of_sample(steps=16)
    X_test = X_test.merge(onpromotion_Te,
                          how="left", left_index=True, right_index=True)
    X_test = X_test.reset_index()
    X_test = X_test.rename(columns={"index": "date"})  # Indexprocessing stage
    # X_test.set_index(['date'])
    X_test = X_test.merge(X_variable_test, how="left", on="date")
    X_test = X_test.set_index(["date"])
    X_test_r.append(X_test)

In [ ]:
X

In [ ]:
X_test

In [ ]:
X_test_r[1].shape

In [ ]:
ridge = make_pipeline(RobustScaler(), Ridge(alpha=10))

In [ ]:
y_pred_r = []
y_fit_r = []
y_test_r = []
result_test = []

for X_test, X, y in zip(X_test_r, X_arr, y_arr):

    model = ridge.fit(X, y)

    y_fit = pd.DataFrame(model.predict(X),
                         index=X.index,
                         columns=y.columns).clip(0.0)
    y_fit_r.append(y_fit)

    y_test = pd.DataFrame(
        model.predict(X_test),
        index=X_test.index,
        columns=y.columns
    ).clip(0.0)
    y_test_r.append(y_test)
    result_test.append(y_test.stack(["family"]))

In [ ]:
y_fit_r[53]

In [ ]:
y_test_r[53]

In [ ]:
result_test[0]

# Data for submission

In [ ]:
# Get correct dates for submission
dates = [
    "2017-08-16",
    "2017-08-17",
    "2017-08-18",
    "2017-08-19",
    "2017-08-20",
    "2017-08-21",
    "2017-08-22",
    "2017-08-23",
    "2017-08-24",
    "2017-08-25",
    "2017-08-26",
    "2017-08-27",
    "2017-08-28",
    "2017-08-29",
    "2017-08-30",
    "2017-08-31",
]

# Get correct order for submission
order = list(range(1, len(result_test) + 1))
str_map = map(str, order)
correct_order_str = sorted(list(str_map))
int_minus_one = lambda element: int(element) - 1
correct_order_int = list(map(int_minus_one, correct_order_str))

# Create and fill list with predictions in the correct order
data = []
for date in dates:
    for i in correct_order_int:
        data += result_test[i].loc[date].to_list()

# Create dataframe from the list
result = pd.DataFrame(data, columns=["sales"])

In [ ]:
result

In [ ]:
store_salesTest = pd.read_csv(download_dir / "test.csv", usecols=["id"])

In [ ]:
store_salesTest.shape  # same number of lines as the result

In [ ]:
result

In [ ]:
y_submit = store_salesTest.join(result)

In [ ]:
y_submit

In [ ]:
# Just observation, predict Result

In [ ]:
store_salesTest = pd.read_csv(
    download_dir / "test.csv",
    dtype={"store_nbr": "category",
           "family": "category",
           "onpromotion": "uint32"},
    parse_dates=["date"],
    infer_datetime_format=True,
)

In [ ]:
pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_colwidth", None)

In [ ]:
Observ_data = store_salesTest.join(result)

In [ ]:
food_sales = (
    Observ_data.groupby(by=["family", "date"])
    .agg({"sales": "mean"})
    .reset_index()
    .sort_values(by="date", ascending=True)
)

In [ ]:
family_salesx = food_sales.groupby(["family", "date"]).mean().unstack("family")

We will observe the sales rates of the grocery store (all of  stores)

In [ ]:
GROCERY = family_salesx.sales["GROCERY I"]

In [ ]:
GROCERY

In [ ]:
ax = GROCERY.plot(**plot_params)

On a 15-day prediction scale, we can see a higher variability in the sales rate on Sunday for GROCERY I 

Comparison with the real values (the previous week)

In [ ]:
fa_sa = (
    dt_Train.groupby(by=["family", "date"])
    .agg({"sales": "mean"})
    .reset_index()
    .sort_values(by="date", ascending=True)
)
mask = (fa_sa["date"] >= "2017-08-01") & (fa_sa["date"] <= "2017-08-15")

fa_sa = fa_sa.loc[mask]
family_sales2 = fa_sa.groupby(["family", "date"]).mean().unstack("family")
food_sales1 = family_sales2.sales["GROCERY I"]

In [ ]:
ax = food_sales1.plot(**plot_params)

Here, we can observe the same variations on the type of product "Grocery". We have well predicted the Test data